<a href="https://colab.research.google.com/github/hgdsraj/AI-Facial-Feature-Detection/blob/master/cmpt_414_final_project_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
from time import sleep
from google.colab import drive
from tensorflow.keras.layers import PReLU, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras import regularizers

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.listdir('/content/drive/My Drive/417data')

['SampleSubmission.csv',
 'IdLookupTable.csv',
 'test.csv',
 'training.csv',
 'val_acc_test.txt',
 'training_mean.csv',
 'old_model.json',
 'good_model.h5',
 'good_model.json',
 'old_model.h5',
 'val_acc.txt',
 'model.json',
 'model.h5']

In [0]:
Train_Dir = '/content/drive/My Drive/417data/training.csv'
Test_Dir = '/content/drive/My Drive/417data/test.csv'
Lookup_Table_Dir = '/content/drive/My Drive/417data/IdLookupTable.csv'
train_data = pd.read_csv(Train_Dir)  
lookup_table_data = pd.read_csv(Lookup_Table_Dir)


In [0]:
#check for missing values
train_data.isnull().any().value_counts()


True     28
False     3
dtype: int64

In [0]:
# fill missing data with frontfill -> then backfill 
train_data.fillna(method = 'ffill',inplace = True)
train_data.fillna(method = 'bfill',inplace = True)

train_data.isnull().any().value_counts() # ensure none remain

False    31
dtype: int64

In [0]:
imag = []
for i in range(0,len(train_data['Image'])):
    img = train_data['Image'][i].split(' ')
    img = ['0' if x == '' else x for x in img]
    imag.append(img)

In [0]:
#reshape to 96 by 96 float (img size)
image_list = np.array(imag,dtype = 'float')
X_train = image_list.reshape(-1,96,96,1)

In [0]:
training = train_data.drop('Image',axis = 1)

y_train = []
for i in range(0,len(train_data['Image'])):
    y = training.iloc[i,:]

    y_train.append(y)
y_train = np.array(y_train,dtype = 'float')

In [0]:
# """TRAIN VALIDATION SPLIT"""
# y_validation = y_train[-1409:]
# y_train = y_train[:-1409]

# X_validation = X_train[-1409:]
# X_train = X_train[:-1409]

In [0]:
model = Sequential()

model.add(BatchNormalization(input_shape=(96, 96, 1)))

model.add(Conv2D(32, (3,3), padding='same', kernel_initializer="he_normal", use_bias=False, input_shape=(96,96,1)))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l1_l2(l1 = 0.0005, l2=0.00005)))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l1_l2(l1 = 0.0005, l2=0.00005)))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), padding='same', use_bias=False, kernel_regularizer=regularizers.l1_l2(l1 = 0.0005, l2=0.00005)))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2, 2), padding="same"))

model.add(Conv2D(64, (3,3), padding='same', use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same', use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same', use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same', use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2, 2), padding="valid"))

model.add(Conv2D(96, (3,3), padding='same', use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Conv2D(96, (3,3), padding='same', use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2, 2), padding="valid"))

model.add(Conv2D(128, (3,3),padding='same', use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3),padding='same', use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Dropout(0.1))

model.add(MaxPool2D(pool_size=(2, 2), padding="same"))

model.add(Conv2D(256, (3,3),padding='same',use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3),padding='same',use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Dropout(0.1))

model.add(MaxPool2D(pool_size=(2, 2), padding="same"))

model.add(Conv2D(512, (3,3), padding='same', use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())

model.add(Conv2D(512, (3,3), padding='same', use_bias=False))
model.add(PReLU(alpha_initializer='zeros'))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu', kernel_regularizer=regularizers.l1_l2(l1 = 0.0005, l2=0.00005)))
model.add(Dropout(0.1))
model.add(Dense(90,activation='relu'))

model.add(Dense(30))
model.summary()




Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 96, 96, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 32)        288       
_________________________________________________________________
p_re_lu (PReLU)              (None, 96, 96, 32)        294912    
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 32)        9216      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 96, 96, 32)        294912    
_________________________________________________________________
batch_normalization_2 (Batch (None, 96, 96, 32)        1

In [0]:
model.compile(optimizer='adam', 
              loss='mse', # TODO mean_squared_error ? 
              metrics=['mae', 'accuracy']) 

In [0]:
history = model.fit(X_train,y_train,epochs = 400, batch_size = 256, validation_split=0.2, use_multiprocessing = True, workers=10)

Epoch 1/400
23/23 [==============================] - 10s 418ms/step - loss: 675.6827 - mae: 18.5121 - accuracy: 0.0679 - val_loss: 121.3105 - val_mae: 8.3732 - val_accuracy: 0.3121
Epoch 2/400
23/23 [==============================] - 9s 374ms/step - loss: 57.3406 - mae: 4.7069 - accuracy: 0.4494 - val_loss: 45.4066 - val_mae: 3.8500 - val_accuracy: 0.6872
Epoch 3/400
23/23 [==============================] - 9s 374ms/step - loss: 43.5526 - mae: 3.7118 - accuracy: 0.4260 - val_loss: 35.6604 - val_mae: 3.1770 - val_accuracy: 0.6872
Epoch 4/400
23/23 [==============================] - 9s 373ms/step - loss: 40.5417 - mae: 3.5648 - accuracy: 0.4613 - val_loss: 25.0835 - val_mae: 1.7013 - val_accuracy: 0.6872
Epoch 5/400
23/23 [==============================] - 9s 373ms/step - loss: 36.1748 - mae: 3.2285 - accuracy: 0.4747 - val_loss: 30.2493 - val_mae: 2.8001 - val_accuracy: 0.6872
Epoch 6/400
23/23 [==============================] - 9s 373ms/step - loss: 34.8376 - mae: 3.1619 - accuracy: 0.

KeyboardInterrupt: ignored

In [0]:
with open("/content/drive/My Drive/417data/val_acc.txt", "w") as txt:
    txt.write(str(history.history['val_accuracy']))
print(history.history['val_accuracy'])


In [0]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# summarize history for error
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('Model Error')
plt.ylabel('mean absolute error')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("/content/drive/My Drive/417data/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/417data/model.h5")
print("Saved model to disk")

In [0]:
# from keras.utils import plot_model
# plot_model(model, to_file='/content/drive/My Drive/417data/model.png')
